In [1]:
from osgeo import gdal,ogr
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:

datapath = 'E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\DataFile\\Mosaic_clip'
Spectral =  'E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Spectral_Indices\\'


if not os.path.exists(Spectral):
        os.makedirs(Spectral) 
dates = []
images = {}
temp = {}

for root, dirs, files in os.walk(datapath):
    for file in files:
        date = file.split('_')[0]
        if date not in dates:
            dates.append(date)
            images[date] = [f"{root}\\{file}"]
            temp[date] = [file]
        else:
            images[date].append(f"{root}\\{file}")
            temp[date].append(file)
# print(images)

for k in temp.keys():
    print(f"{k}  {temp[k]}")
    

2020-03-27  ['2020-03-27_mosaic_clip.tif', '2020-03-27_udm_mosaic_clip.tif']
2020-04-30  ['2020-04-30_mosaic_clip.tif', '2020-04-30_udm_mosaic_clip.tif']
2020-05-15  ['2020-05-15_mosaic_clip.tif', '2020-05-15_udm_mosaic_clip.tif']
2020-10-11  ['2020-10-11_mosaic_clip.tif', '2020-10-11_udm_mosaic_clip.tif']
2020-10-15  ['2020-10-15_mosaic_clip.tif', '2020-10-15_udm_mosaic_clip.tif']
2020-11-07  ['2020-11-07_mosaic_clip.tif', '2020-11-07_udm_mosaic_clip.tif']
2020-11-09  ['2020-11-09_mosaic_clip.tif', '2020-11-09_udm_mosaic_clip.tif']
2020-11-28  ['2020-11-28_mosaic_clip.tif', '2020-11-28_udm_mosaic_clip.tif']
2020-11-29  ['2020-11-29_mosaic_clip.tif', '2020-11-29_udm_mosaic_clip.tif']
2020-12-27  ['2020-12-27_mosaic_clip.tif', '2020-12-27_udm_mosaic_clip.tif']
2020-12-28  ['2020-12-28_mosaic_clip.tif', '2020-12-28_udm_mosaic_clip.tif']
2020-12-31  ['2020-12-31_mosaic_clip.tif', '2020-12-31_udm_mosaic_clip.tif']
2021-01-10  ['2021-01-10_mosaic_clip.tif', '2021-01-10_udm_mosaic_clip.tif']

8-Band asset band order¶
The 8-band multispectral band order is:

Band 1 = Coastal Blue
Band 2 = Blue
Band 3 = Green I
Band 4 = Green
Band 5 = Yellow
Band 6 = Red
Band 7 = Red Edge
Band 8 = Near-infrared

Calculations of spectral indices¶
The spectral indices are calculated using the following formulae:




In [3]:
# Vegetation Indices
def NDVI(NIR, Red):
    return (NIR - Red) / (NIR + Red)

def SAVI(NIR, Red, L):
    return ((NIR - Red) / (NIR + Red + L)) * (1 + L)

def EVI(NIR, Red, Blue, G, C1, C2, L):
    return G * ((NIR - Red) / (NIR + C1 * Red - C2 * Blue + L))

def GNDVI(NIR, Green):
    return (NIR - Green) / (NIR + Green)

def TSAVI(NIR, Red, a, s, X):
    return (NIR - Red - a) / (NIR + Red - a * s + X * (1 + s**2))

# Water Indices
def NDWI(Green, NIR):
    return (Green - NIR) / (Green + NIR)

def MNDWI(Green, SWIR):
    return (Green - SWIR) / (Green + SWIR)

def AWEI(Green, NIR, SWIR1, SWIR2):
    return 4 * (Green - SWIR2) - (0.25 * NIR + 2.75 * SWIR1)

def WRI(Green, NIR):
    return Green / NIR

def LWBI(NIR, Green):
    return NIR / Green

# Open Mud Indices
def NDSI(Green, SWIR):
    return (Green - SWIR) / (Green + SWIR)

def MSI(NIR, SWIR):
    return NIR / SWIR

def NDCMI(SWIR1, SWIR2):
    return (SWIR1 - SWIR2) / (SWIR1 + SWIR2)

def NDSI(NIR, Red):
    return (NIR - Red) / (NIR + Red)

def BI(Red, Green, NIR, SWIR):
    return (Red**2 + Green**2 + NIR**2 + SWIR**2)**0.5

# Buildings/Human-made objects Indices
def NDBI(SWIR, NIR):
    return (SWIR - NIR) / (SWIR + NIR)

def BAI(Red, NIR):
    return 1 / ((0.1 - Red)**2 + (0.06 - NIR)**2)

def UI(NIR, Red):
    return (NIR - Red) / (NIR + Red)

def HSI(NIR, Red, Green, SWIR):
    return ((NIR - Red) / (NIR + Red)) - ((Green - SWIR) / (Green + SWIR))

def MNDBI(SWIR, NIR, Green):
    return ((SWIR - NIR) / (SWIR + NIR)) - ((Green - SWIR) / (Green + SWIR))

# Coral Reef in the water Indices
def CHI(Green, Red):
    return (Green - Red) / (Green + Red)

def CRI(Green, Blue):
    return (Green - Blue) / (Green + Blue)

def CBI(NIR, Red):
    return (NIR - Red) / (NIR + Red)

def CPI(NIR, Green):
    return (NIR - Green) / (NIR + Green)

def CRI(Blue, Green):
    return (Blue - Green) / (Blue + Green)



In [4]:
L = 0.5
G = 2.5
C1 = 6
C2 = 7.5
a = 0.16
s = 0.5
X = 0.1

In [ ]:
# Save Spectral Indices into a new file
dst_ds = None
for date in dates:
    print(f"Processing {date}")
    for i in range(len(images[date])):
        if images[date][i].endswith('udm_mosaic_clip.tif'):
            continue
        print(f"Processing {images[date][i]}")
        raster = rasterio.open(images[date][i])
        top_left_x = raster.bounds.left
        top_left_y = raster.bounds.top
        pixel_width = raster.res[0]
        pixel_height = raster.res[1]
        
        geotransform = [top_left_x, pixel_width, 0, top_left_y, 0, -pixel_height]
        
        
        NIR = raster.read(8)
        Red = raster.read(6)
        Blue = raster.read(2)
        Green = raster.read(4)
        SWIR = raster.read(7)
        SWIR1 = raster.read(5)
        SWIR2 = raster.read(6)
        
        NDVI_image = NDVI(NIR, Red)
        SAVI_image = SAVI(NIR, Red, L)
        EVI_image = EVI(NIR, Red, Blue, G, C1, C2, L)
        GNDVI_image = GNDVI(NIR, Green)
        TSAVI_image = TSAVI(NIR, Red, a, s, X)
        
        NDWI_image = NDWI(Green, NIR)
        MNDWI_image = MNDWI(Green, SWIR)
        AWEI_image = AWEI(Green, NIR, SWIR1, SWIR2)
        WRI_image = WRI(Green, NIR)
        LWBI_image = LWBI(NIR, Green)
        
        NDSI_image = NDSI(Green, SWIR)
        MSI_image = MSI(NIR, SWIR)
        NDCMI_image = NDCMI(SWIR1, SWIR2)
        NDSI_image = NDSI(NIR, Red)
        BI_image = BI(Red, Green, NIR, SWIR)
        
        NDBI_image = NDBI(SWIR, NIR)
        BAI_image = BAI(Red, NIR)
        UI_image = UI(NIR, Red)
        HSI_image = HSI(NIR, Red, Green, SWIR)
        MNDBI_image = MNDBI(SWIR, NIR, Green)
        
        CHI_image = CHI(Green, Red)
        CRI_image = CRI(Green, Blue)
        CBI_image = CBI(NIR, Red)
        CPI_image = CPI(NIR, Green)
        CRI_image = CRI(Blue, Green)
        
        # Save the Spectral Indices into a new file
        output = f"{Spectral}{date}_spectral_indices.tif"
        print(f"Saving {output}")
        driver = gdal.GetDriverByName('GTiff')
        
        dst_ds = driver.Create(output, raster.width, raster.height, 25, gdal.GDT_Float32)
        dst_ds.SetGeoTransform(geotransform)
        
        
        dst_ds.SetProjection(raster.crs.to_wkt())
        
        dst_ds.GetRasterBand(1).WriteArray(NDVI_image)
        dst_ds.GetRasterBand(1).description = 'NDVI'
        dst_ds.GetRasterBand(2).WriteArray(SAVI_image)
        dst_ds.GetRasterBand(2).description = 'SAVI'
        dst_ds.GetRasterBand(3).WriteArray(EVI_image)
        dst_ds.GetRasterBand(3).description = 'EVI'
        dst_ds.GetRasterBand(4).WriteArray(GNDVI_image)
        dst_ds.GetRasterBand(4).description = 'GNDVI'
        dst_ds.GetRasterBand(5).WriteArray(TSAVI_image)
        dst_ds.GetRasterBand(5).description = 'TSAVI'
        
        dst_ds.GetRasterBand(6).WriteArray(NDWI_image)
        dst_ds.GetRasterBand(6).description = 'NDWI'
        dst_ds.GetRasterBand(7).WriteArray(MNDWI_image)
        dst_ds.GetRasterBand(7).description = 'MNDWI'
        dst_ds.GetRasterBand(8).WriteArray(AWEI_image)
        dst_ds.GetRasterBand(8).description = 'AWEI'
        dst_ds.GetRasterBand(9).WriteArray(WRI_image)
        dst_ds.GetRasterBand(9).description = 'WRI'
        dst_ds.GetRasterBand(10).WriteArray(LWBI_image)
        dst_ds.GetRasterBand(10).description = 'LWBI'
        
        dst_ds.GetRasterBand(11).WriteArray(NDSI_image)
        dst_ds.GetRasterBand(11).description = 'NDSI'
        dst_ds.GetRasterBand(12).WriteArray(MSI_image)
        dst_ds.GetRasterBand(12).description = 'MSI'
        dst_ds.GetRasterBand(13).WriteArray(NDCMI_image)
        dst_ds.GetRasterBand(13).description = 'NDCMI'
        dst_ds.GetRasterBand(14).WriteArray(NDSI_image)
        dst_ds.GetRasterBand(14).description = 'NDSI'
        dst_ds.GetRasterBand(15).WriteArray(BI_image)
        dst_ds.GetRasterBand(15).description = 'BI'
        
        dst_ds.GetRasterBand(16).WriteArray(NDBI_image)
        dst_ds.GetRasterBand(16).description = 'NDBI'
        dst_ds.GetRasterBand(17).WriteArray(BAI_image)
        dst_ds.GetRasterBand(17).description = 'BAI'
        dst_ds.GetRasterBand(18).WriteArray(UI_image)
        dst_ds.GetRasterBand(18).description = 'UI'
        dst_ds.GetRasterBand(19).WriteArray(HSI_image)
        dst_ds.GetRasterBand(19).description = 'HSI'
        dst_ds.GetRasterBand(20).WriteArray(MNDBI_image)
        dst_ds.GetRasterBand(20).description = 'MNDBI'
        
        dst_ds.GetRasterBand(21).WriteArray(CHI_image)
        dst_ds.GetRasterBand(21).description = 'CHI'
        dst_ds.GetRasterBand(22).WriteArray(CRI_image)
        dst_ds.GetRasterBand(22).description = 'CRI'
        dst_ds.GetRasterBand(23).WriteArray(CBI_image)
        dst_ds.GetRasterBand(23).description = 'CBI'
        dst_ds.GetRasterBand(24).WriteArray(CPI_image)
        dst_ds.GetRasterBand(24).description = 'CPI'
        dst_ds.GetRasterBand(25).WriteArray(CRI_image)
        dst_ds.GetRasterBand(25).description = 'CRI'
        
        dst_ds = None
        print(f"Saved {output}\n")
    

Processing 2020-03-27
Processing E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\DataFile\Mosaic_clip\2020-03-27_mosaic_clip.tif
Saving E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Spectral_Indices\2020-03-27_spectral_indices.tif
Saved E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Spectral_Indices\2020-03-27_spectral_indices.tif

Processing 2020-04-30
Processing E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\DataFile\Mosaic_clip\2020-04-30_mosaic_clip.tif
Saving E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Spectral_Indices\2020-04-30_spectral_indices.tif
Saved E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Spectral_Indices\2020-04-30_spectral_indices.tif

Processing 2020-05-15
Processing E:\LULC Change Detection Using Unsupervised Clust

In [ ]:
# Plot the Spectral Indices        
